<a href="https://colab.research.google.com/github/Anudeep003/Codtech-week4/blob/main/RecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pandas scikit-learn torch

In [4]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [5]:
!wget -q https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -q ml-100k.zip

In [7]:
# Load ratings
df = pd.read_csv('ml-100k/u.data', sep='\t', names=["user_id", "item_id", "rating", "timestamp"])
df = df.drop(columns=["timestamp"])
df

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['user_id'].plot(kind='hist', bins=20, title='user_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['item_id'].plot(kind='hist', bins=20, title='item_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['rating'].plot(kind='hist', bins=20, title='rating')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_10.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_11['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_12['user_id'].plot(kind='line', figsize=(8, 4), title='user_id')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['item_id'].plot(kind='line', figsize=(8, 4), title='item_id')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14['rating'].plot(kind='line', figsize=(8, 4), title='rating')
plt.gca().spines[['top', 'right']].set_visible(False)

In [8]:
user2idx = {user_id: i for i, user_id in enumerate(df['user_id'].unique())}
item2idx = {item_id: i for i, item_id in enumerate(df['item_id'].unique())}

df['user'] = df['user_id'].map(user2idx)
df['item'] = df['item_id'].map(item2idx)

train, test = train_test_split(df[['user', 'item', 'rating']], test_size=0.2)

In [9]:
class MF(nn.Module):
    def __init__(self, num_users, num_items, latent_dim=50):
        super(MF, self).__init__()
        self.user_emb = nn.Embedding(num_users, latent_dim)
        self.item_emb = nn.Embedding(num_items, latent_dim)

    def forward(self, u, i):
        return (self.user_emb(u) * self.item_emb(i)).sum(1)

In [11]:
model = MF(len(user2idx), len(item2idx))
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

u = torch.tensor(train['user'].values)
i = torch.tensor(train['item'].values)
r = torch.tensor(train['rating'].values, dtype=torch.float)

for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    pred = model(u, i)
    loss = loss_fn(pred, r)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}: loss = {loss.item():.4f}")

Epoch 1: loss = 64.5458
Epoch 2: loss = 62.4140
Epoch 3: loss = 60.3556
Epoch 4: loss = 58.3694
Epoch 5: loss = 56.4544
Epoch 6: loss = 54.6090
Epoch 7: loss = 52.8318
Epoch 8: loss = 51.1212
Epoch 9: loss = 49.4755
Epoch 10: loss = 47.8927
Epoch 11: loss = 46.3710
Epoch 12: loss = 44.9084
Epoch 13: loss = 43.5031
Epoch 14: loss = 42.1530
Epoch 15: loss = 40.8560
Epoch 16: loss = 39.6102
Epoch 17: loss = 38.4136
Epoch 18: loss = 37.2641
Epoch 19: loss = 36.1597
Epoch 20: loss = 35.0985
Epoch 21: loss = 34.0786
Epoch 22: loss = 33.0980
Epoch 23: loss = 32.1548
Epoch 24: loss = 31.2473
Epoch 25: loss = 30.3735
Epoch 26: loss = 29.5319
Epoch 27: loss = 28.7207
Epoch 28: loss = 27.9382
Epoch 29: loss = 27.1828
Epoch 30: loss = 26.4531
Epoch 31: loss = 25.7475
Epoch 32: loss = 25.0646
Epoch 33: loss = 24.4031
Epoch 34: loss = 23.7615
Epoch 35: loss = 23.1387
Epoch 36: loss = 22.5334
Epoch 37: loss = 21.9445
Epoch 38: loss = 21.3709
Epoch 39: loss = 20.8117
Epoch 40: loss = 20.2658
Epoch 41:

In [13]:
movie_titles = {}

with open('ml-100k/u.item', encoding='ISO-8859-1') as f:
    for line in f:
        parts = line.strip().split('|')
        movie_id = int(parts[0])
        title = parts[1]
        movie_titles[movie_id] = title

In [14]:
def recommend(user_id, top_k=5):
    model.eval()
    u_idx = torch.tensor([user2idx[user_id]] * len(item2idx))
    i_idx = torch.tensor(list(item2idx.values()))
    scores = model(u_idx, i_idx).detach().numpy()

    top_items = sorted(zip(item2idx.keys(), scores), key=lambda x: -x[1])[:top_k]
    recommended_ids = [item for item, _ in top_items]

    recommended_names = [movie_titles.get(movie_id, f"Movie ID {movie_id}") for movie_id in recommended_ids]
    return recommended_names

In [17]:
print("Recommended movies for user 42:")
for title in recommend(user_id=42):
    print("🎬", title)

Recommended movies for user 42:
🎬 For Ever Mozart (1996)
🎬 Spanking the Monkey (1994)
🎬 Cowboy Way, The (1994)
🎬 Star Maps (1997)
🎬 Chairman of the Board (1998)
